In [1]:
import utils.Evaluation as ev
import utils.CustomModel as cm
import utils.Encryption as en
import utils.utils as ut
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import json

d:\ProgramData\Anaconda3\envs\isic\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
root_dir = "F:\\pps_paper\\dataset"
train_dir = f"{root_dir}\\train"
test_dir = f"{root_dir}\\test"
validation_dir = f"{root_dir}\\validation"

train_images = f"{root_dir}\\train\\DataE"
train_masks = f"{root_dir}\\train\\GroundTruth_1E"
test_images = f"{root_dir}\\test\\DataE"
test_masks = f"{root_dir}\\test\\GroundTruth_1E"
validation_images = f"{root_dir}\\validation\\DataE"
validation_masks = f"{root_dir}\\validation\\GroundTruth_1E"

In [3]:
image_name = 'ISIC_0000001' 
img = cv2.imread(f'F:\\pps_paper\\dataset\\train\\DataE\\{image_name}.jpg',1)
groundTruth = cv2.imread(f'F:\\pps_paper\\dataset\\train\\GroundTruth_1E\\{image_name}_segmentation.png',0)


In [4]:
image_name = 'ISIC_0000376' 
img = cv2.imread(f'F:\\pps_paper\\dataset\\train\\DataE\\{image_name}.jpg',1)
groundTruth = cv2.imread(f'F:\\pps_paper\\dataset\\train\\GroundTruth_1E\\{image_name}_segmentation.png',0)
key = None
with open('key.json','r') as fp:
    key = json.load(fp)

Zb , Zg , Zr = cv2.split(img)
Zb , Zg , Zr = Zb.flatten() , Zg.flatten() , Zr.flatten()  
# convert to np.float32
Zr = np.float32(Zr)
# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 2

ret,label,center = cv2.kmeans(Zr,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
# Now convert back into uint8, and make original image
center = np.uint8(center)
red = center[label.flatten()]
red = red.reshape((img.shape[0],img.shape[1]))

red_un = en.unscramble(red,key)

In [8]:
gt = tf.convert_to_tensor(groundTruth)
result = tf.convert_to_tensor((red_un < 200)*255)
result = result / tf.reduce_max(result)


In [9]:
conf_mat = ev.get_confusion_matrix(gt, result)

In [ ]:
plt.imshow((red_un < 200)*255, cmap='gray')